In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, RepeatedKFold
from sklearn.metrics import make_scorer, mean_squared_error
from typing import Tuple

In [3]:
tr = '../Data/train.csv'
ts = '../Data/test.csv'
pred_cols = ["Mean_BMI","Median_BMI","Unmet_Need_Rate","Under5_Mortality_Rate","Skilled_Birth_Attendant_Rate","Stunted_Rate"]

In [4]:
df = pd.read_csv(tr)
df

,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate,0,1,2,3,...,718,719,720,721,722,723,724,725,726,727
0,24.12,25.28,50.00,9.68,100.00,20.00,-17.125811,-34.640745,14.759849,-30.167421,...,0.552191,0.141088,0.222730,-0.252435,-1.080846,-0.217270,0.946620,-0.001225,1.013862,-0.035631
1,23.04,21.98,7.69,8.33,66.67,0.00,-17.076751,-35.178017,14.140446,-29.893448,...,1.050924,-0.096601,1.081103,-0.324452,-1.040685,0.250936,0.968019,-0.245824,0.743797,-0.297141
2,26.74,26.57,7.69,2.86,100.00,0.00,-17.169738,-34.444535,14.395977,-30.300743,...,0.936866,0.350209,0.435342,-0.362765,-0.539886,-0.531760,0.124996,0.189406,0.600528,-0.168457
3,27.58,28.08,0.00,9.52,NaN,0.00,-17.396199,-34.928404,15.799118,-32.392487,...,0.615778,-0.271387,0.459463,-0.381200,-0.499744,-0.126459,0.324357,0.354043,0.210230,-0.291572
4,24.23,23.77,20.00,23.81,50.00,0.00,-17.418952,-35.566681,17.605409,-30.432972,...,0.587977,-0.482609,0.538158,0.160229,-0.618283,-0.217712,0.441060,-0.055342,0.372976,-0.359658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99106,21.92,21.08,6.25,9.72,40.00,21.74,-12.502575,-2.109966,-13.474339,34.101718,...,0.311839,1.208883,-0.298171,-0.613468,0.196826,0.064507,0.279913,-0.049750,1.093151,0.026888
99107,23.16,22.14,33.33,2.04,78.95,25.00,-12.298399,-1.751321,-14.694330,34.207399,...,-0.816502,0.717558,0.111479,-0.031346,-0.286870,0.044142,-0.083391,-0.196787,0.234070,-0.038043
99108,24.33,22.61,11.11,1.75,76.19,22.22,-12.383431,-1.557209,-15.049148,33.343708,...,-0.706379,1.159263,0.032595,-0.462457,-0.506553,0.319599,0.438168,-0.231995,1.114789,0.020124
99109,23.70,21.44,10.53,4.88,68.00,17.65,-12.324838,-1.622992,-14.939852,33.843820,...,-0.605190,0.768277,0.165389,-0.209598,0.011767,0.116503,0.061010,-0.330933,0.597309,0.182588


In [5]:
y = df[pred_cols]
X = df.drop(pred_cols, axis=1)

In [6]:
# from sklearn.impute import KNNImputer

# imputer = KNNImputer()
# y_hat = imputer.fit_transform(y)

In [7]:
# y_hat = pd.DataFrame(y_hat)
y_hat = pd.read_csv('../Data/y_hat.csv')
y_hat.isnull().sum()

Mean_BMI                        0
Median_BMI                      0
Unmet_Need_Rate                 0
Under5_Mortality_Rate           0
Skilled_Birth_Attendant_Rate    0
Stunted_Rate                    0
dtype: int64

In [17]:
def mcrmse(predt: np.ndarray, dtrain: xgb.DMatrix) -> Tuple[str, float]:
    ''' Mean column-wise root mean square error metric.'''
    y = dtrain.get_label().reshape(-1, 6)
    # loop over each column and calculate error
    errors = []
    for i in range(predt.shape[1]):
        errors.append(np.sqrt(mean_squared_error(y[:,i], predt[:,i])))
    ans = np.mean(errors)
    return 'MCRMSE', float(ans)

In [18]:
from xgboost import cv

params = {"objective":"reg:squarederror",'colsample_bytree': 0.8,'learning_rate': 0.3,
                'max_depth': 10, 'gamma': 1.5, 'min_child_weight': 5}

xgb_cv = cv(params=params, dtrain=xgb.DMatrix(X, y_hat), num_boost_round=100, nfold=5, custom_metric=mcrmse, early_stopping_rounds=10, seed=123, as_pandas=True)

In [19]:
xgb_cv

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std,train-MCRMSE-mean,train-MCRMSE-std,test-MCRMSE-mean,test-MCRMSE-std
0,30.296484,0.008179,30.622479,0.024762,26.160909,0.005650,26.425821,0.016565
1,24.418486,0.017140,25.174628,0.016593,20.951142,0.013407,21.551465,0.016407
2,20.738397,0.030128,21.984468,0.030446,17.594517,0.020596,18.571478,0.024432
3,18.419204,0.046053,20.231507,0.039130,15.410896,0.031939,16.819368,0.029752
4,16.953720,0.041172,19.311044,0.047420,13.989183,0.029732,15.816524,0.033938
5,15.989431,0.037506,18.841783,0.045642,13.036645,0.029365,15.259817,0.031287
6,15.278577,0.042148,18.619644,0.049823,12.349190,0.030788,14.969360,0.034369
7,14.726274,0.045861,18.515163,0.048626,11.832779,0.034011,14.822763,0.032715
8,14.242933,0.052159,18.482510,0.050216,11.405275,0.038860,14.760672,0.033485
9,13.831596,0.045914,18.475313,0.054133,11.047905,0.036808,14.737540,0.036090


In [22]:
model = xgb.XGBRegressor(objective="reg:squarederror", colsample_bytree=0.8, 
                         learning_rate=0.3, max_depth=10, 
                         gamma=1.5, min_child_weight=5, 
                         eval_metric=mcrmse, verbosity=2)
model.fit(X, y_hat)

[11:47:03] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 130 extra nodes, 0 pruned nodes, max_depth=10
[11:47:05] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 146 extra nodes, 0 pruned nodes, max_depth=10
[11:47:08] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 1424 extra nodes, 0 pruned nodes, max_depth=10
[11:47:12] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 1246 extra nodes, 4 pruned nodes, max_depth=10
[11:47:15] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end,

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False,
             eval_metric=<function mcrmse at 0x16d3f00d0>, feature_types=None,
             gamma=1.5, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.3, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [26]:
pred = model.predict(pd.read_csv(ts))
pred = pd.DataFrame(pred, columns=pred_cols)
ind = pd.read_csv('../Data/sample submission.csv')['DHSID']
pred['DHSID'] = ind
pred = pred[['DHSID','Mean_BMI','Median_BMI','Unmet_Need_Rate','Under5_Mortality_Rate','Skilled_Birth_Attendant_Rate','Stunted_Rate']]
pred

,DHSID,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
0,AL200800000003,25.976192,24.493462,43.833202,3.779210,103.534988,2.698278
1,AL200800000005,25.510635,24.274622,40.865852,7.760945,90.451378,13.785562
2,AL200800000007,24.531858,23.785955,30.039881,5.027587,82.706848,6.588806
3,AL200800000008,24.504065,24.902952,45.748093,6.705809,95.424690,-1.390617
4,AL200800000009,26.085123,25.342991,28.347229,10.694656,76.905731,6.255195
...,...,...,...,...,...,...,...
14995,ZW201500000382,24.680107,20.821726,0.178756,8.667793,48.517410,32.516144
14996,ZW201500000383,22.293596,21.457970,12.169876,6.318041,34.340851,27.659513
14997,ZW201500000386,21.989826,22.956793,18.320070,6.631188,32.695278,45.633564
14998,ZW201500000390,21.882816,22.211367,11.675657,7.709791,43.243607,45.514191


In [27]:
pred.to_csv('../submission/submission_new.csv', index=False)

In [8]:
# model = xgb.XGBRegressor(objective="reg:squarederror", feval=mcrmse)

In [9]:
# params = {
#         'eta': [0.1, 0.3],
#         'min_child_weight': [1, 5, 10],
#         'gamma': [0.5, 1, 1.5, 2, 5],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.6, 0.8, 1.0],
#         'max_depth': [3, 4, 5, 10, 15]
#         }

In [ ]:
# folds = 15
# param_comb = 200

# rkf = RepeatedKFold(n_splits=folds, random_state = 1001)

# random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=param_comb, 
#                                    scoring=make_scorer(mcrmse), cv=rkf, 
#                                    verbose=3, random_state=1001)

# # Here we go
# # start_time = timer(None) # timing starts from this point for "start_time" variable
# random_search.fit(X, y_hat)
# # timer(start_time) # timing ends here for "start_time" variable